In [1]:
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf

In [ ]:
train_dir = r'C:\Users\tusha\Desktop\Major Project\Flask_app\chest_xray\train'
val_dir = r'C:\Users\tusha\Desktop\Major Project\Flask_app\chest_xray\val'
test_dir = r'C:\Users\tusha\Desktop\Major Project\Flask_app\chest_xray\test'

In [ ]:
IMG_HEIGHT = 224
IMG_WIDTH = 224

BATCH_SIZE = 32

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255
)

In [ ]:
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='binary',
    batch_size=BATCH_SIZE
)

val_data = train_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='binary',
    batch_size=BATCH_SIZE
)

test_data = train_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='binary',
    batch_size=BATCH_SIZE
)

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
mobilenet = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

mobilenet.trainable = False

In [ ]:
inputs = tf.keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))

pretrained_model = mobilenet(inputs, training=False)

dense = tf.keras.layers.Dense(1024, activation='relu')(pretrained_model)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense)


model = tf.keras.Model(inputs, outputs)


print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 1280)             2257984   
 ional)                                                          
                                                                 
 dense (Dense)               (None, 1024)              1311744   
                                                                 
 dense_1 (Dense)             (None, 1)                 1025      
                                                                 
Total params: 3,570,753
Trainable params: 1,312,769
Non-trainable params: 2,257,984
_________________________________________________________________
None


In [ ]:
EPOCHS = 50

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.AUC(name='auc')
    ]
)

history = model.fit(
    train_data,
    validation_data=val_data,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

Epoch 1/50
163/163 [==============================] - 221s 1s/step - loss: 0.2187 - accuracy: 0.9220 - auc: 0.9609 - val_loss: 0.2755 - val_accuracy: 0.8125 - val_auc: 0.9688
Epoch 2/50
163/163 [==============================] - 211s 1s/step - loss: 0.1139 - accuracy: 0.9549 - auc: 0.9893 - val_loss: 0.6211 - val_accuracy: 0.7500 - val_auc: 0.9922
Epoch 3/50
163/163 [==============================] - 222s 1s/step - loss: 0.1108 - accuracy: 0.9540 - auc: 0.9892 - val_loss: 0.0850 - val_accuracy: 1.0000 - val_auc: 1.0000
Epoch 4/50
163/163 [==============================] - 250s 2s/step - loss: 0.0797 - accuracy: 0.9688 - auc: 0.9940 - val_loss: 0.3923 - val_accuracy: 0.7500 - val_auc: 1.0000
Epoch 5/50
163/163 [==============================] - 224s 1s/step - loss: 0.0851 - accuracy: 0.9672 - auc: 0.9935 - val_loss: 0.2782 - val_accuracy: 0.8750 - val_auc: 0.9688
Epoch 6/50
163/163 [==============================] - 221s 1s/step - loss: 0.0831 - accuracy: 0.9666 - auc: 0.9945 - val_loss

In [ ]:
from sklearn.metrics import confusion_matrix

true_labels = test_data.labels
pred_labels = np.squeeze(np.array(model.predict(test_data) >= 0.5, dtype=np.int))

cm = confusion_matrix(true_labels, pred_labels) 

In [ ]:
test_data.class_indices

{'normal': 0, 'pneumonia': 1}

In [ ]:
results = model.evaluate(test_data, verbose=0)

accuracy = results[1]
auc = results[2]

In [ ]:
tn, fp, fn, tp = cm.ravel()

precision = tp / (tp + fp)
recall = tp / (tp + fn)

In [ ]:
print("Accuracy: {:.2f}".format(accuracy))
print("AUC: {:.2f}".format(auc))
print("Precision: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))

Accuracy: 0.82
AUC: 0.93
Precision: 0.64
Recall: 0.83


In [ ]:
model.save("model.h5")